In [17]:
from datetime import datetime
import math
import cv2
import glob
import numpy as np
from numpy import zeros, ones, vstack, hstack
from numpy.random import permutation
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.utils import shuffle
from skimage.metrics import structural_similarity as ssim
import optuna

In [3]:
!pip install opendatasets --quiet,
import opendatasets as od
od.download("https://www.kaggle.com/datasets/puneet6060/intel-image-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: fratzcan
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification


100%|██████████| 346M/346M [00:03<00:00, 120MB/s] 


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [4]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: firatozc (firatozc-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
def RGB2LAB2(R0, G0, B0):
    R = R0 / 255
    G = G0 / 255
    B = B0 / 255

    Y = 0.299*R + 0.587*G + 0.114*B
    X = 0.449*R + 0.353*G + 0.198*B
    Z = 0.012*R + 0.089*G + 0.899*B

    L = Y
    a = (X - Y) / 0.234
    b = (Y - Z) / 0.785

    return L, a, b


def LAB22RGB(L, a, b, device):
    a11, a12, a13 = 0.299, 0.587, 0.114
    a21, a22, a23 = (0.15/0.234), (-0.234/0.234), (0.084/0.234)
    a31, a32, a33 = (0.287/0.785), (0.498/0.785), (-0.785/0.785)

    aa_np = np.array([[a11, a12, a13], [a21, a22, a23], [a31, a32, a33]])
    aa_inv_tensor = torch.tensor(np.linalg.inv(aa_np), dtype=torch.float32).to(device)

    Lab = torch.cat((L, a, b), dim=1) 

    batch_size, _, H, W = Lab.shape

    Lab_reshaped = Lab.view(batch_size, 3, -1)

    Lab_permuted = Lab_reshaped.permute(0, 2, 1) 

    rgb_permuted = torch.matmul(Lab_permuted, aa_inv_tensor.t()) 

    rgb_reshaped = rgb_permuted.permute(0, 2, 1)
    rgb_image = rgb_reshaped.view(batch_size, 3, H, W)

    rgb_image = torch.clamp(rgb_image, 0.0, 1.0)

    return rgb_image

In [6]:
def psnr(img1, img2):
    mse = np.mean((img1.astype("float") - img2.astype("float")) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def mse(imageA, imageB, bands):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1] * bands)
    return err

def mae(imageA, imageB, bands):
    err = np.sum(np.abs((imageA.astype("float") - imageB.astype("float"))))
    err /= float(imageA.shape[0] * imageA.shape[1] * bands)
    return err

def rmse(imageA, imageB, bands):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1] * bands)
    err = np.sqrt(err)
    return err

In [7]:
class DoubleConv(nn.Module):
    """Double Convolution Block"""
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


In [8]:
class TripleConv(nn.Module):
    """Triple Convolution Block"""
    def __init__(self, in_channels, out_channels):
        super(TripleConv, self).__init__()
        self.triple_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.triple_conv(x)


In [ ]:
class UNet1(nn.Module):
    def __init__(self, in_channels=1, out_channels=2):
        super(UNet1, self).__init__()

        # Encoder
        self.conv1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = TripleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)

        self.conv4 = TripleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)

        self.conv5 = TripleConv(512, 512)
        self.pool5 = nn.MaxPool2d(2)

        # Bottleneck
        self.conv55 = TripleConv(512, 512)

        # Decoder
        self.up66 = nn.ConvTranspose2d(512, 512, kernel_size=2, stride=2)
        self.conv66 = DoubleConv(1024, 512)  # 512 + 512 from skip connection

        self.up6 = nn.ConvTranspose2d(512, 512, kernel_size=2, stride=2)
        self.conv6 = DoubleConv(1024, 512)  # 512 + 512 from skip connection

        self.up7 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv7 = DoubleConv(512, 256)  # 256 + 256 from skip connection

        self.up8 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv8 = DoubleConv(256, 128)  # 128 + 128 from skip connection

        self.up9 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv9 = DoubleConv(128, 64)  # 64 + 64 from skip connection

        # Multi-scale feature fusion
        self.up_f02 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.up_f12 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        # Final layers
        self.conv11 = nn.Conv2d(384, 128, kernel_size=3, padding=1)  # 64+64+128+128
        self.relu11 = nn.ReLU(inplace=True)

        self.conv12 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.relu12 = nn.ReLU(inplace=True)

        self.conv13 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.relu13 = nn.ReLU(inplace=True)

        self.conv14 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, x):
        # Encoder
        conv1 = self.conv1(x)
        x1 = self.pool1(conv1)

        conv2 = self.conv2(x1)
        x2 = self.pool2(conv2)

        conv3 = self.conv3(x2)
        x3 = self.pool3(conv3)

        conv4 = self.conv4(x3)
        x4 = self.pool4(conv4)

        conv5 = self.conv5(x4)
        x5 = self.pool5(conv5)

        # Bottleneck
        conv55 = self.conv55(x5)

        # Decoder
        up66 = self.up66(conv55)
        if up66.size()[2:] != conv5.size()[2:]:
            up66 = F.interpolate(up66, size=conv5.size()[2:], mode="bilinear", align_corners=True)
        merge66 = torch.cat([conv5, up66], dim=1)
        conv66 = self.conv66(merge66)

        up6 = self.up6(conv66)
        if up6.size()[2:] != conv4.size()[2:]:
            up6 = F.interpolate(up6, size=conv4.size()[2:], mode="bilinear", align_corners=True)
        merge6 = torch.cat([conv4, up6], dim=1)
        conv6 = self.conv6(merge6)

        up7 = self.up7(conv6)
        if up7.size()[2:] != conv3.size()[2:]:
            up7 = F.interpolate(up7, size=conv3.size()[2:], mode="bilinear", align_corners=True)
        merge7 = torch.cat([conv3, up7], dim=1)
        conv7 = self.conv7(merge7)

        up8 = self.up8(conv7)
        if up8.size()[2:] != conv2.size()[2:]:
            up8 = F.interpolate(up8, size=conv2.size()[2:], mode="bilinear", align_corners=True)
        merge8 = torch.cat([conv2, up8], dim=1)
        conv8 = self.conv8(merge8)

        up9 = self.up9(conv8)
        if up9.size()[2:] != conv1.size()[2:]:
            up9 = F.interpolate(up9, size=conv1.size()[2:], mode="bilinear", align_corners=True)
        merge9 = torch.cat([conv1, up9], dim=1)
        conv9 = self.conv9(merge9)


        # Multi-scale feature fusion
        up_f01 = conv1  
        up_f11 = conv9  
        up_f02 = self.up_f02(conv2)  
        up_f12 = self.up_f12(conv8)  

        merge11 = torch.cat([up_f01, up_f11, up_f02, up_f12], dim=1)

        conv11 = self.relu11(self.conv11(merge11))
        conv12 = self.relu12(self.conv12(conv11))
        conv13 = self.relu13(self.conv13(conv12))
        output = self.tanh(self.conv14(conv13))

        return output

In [ ]:
class ColorizationDataset(Dataset):
    def __init__(self, file_list, dim=150):
        self.file_list = file_list # image paths
        self.dim = dim

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, (self.dim, self.dim))

        sz0, sz1 = img.shape[:2]
        R1 = img[:, :, 0].reshape(-1, 1)
        G1 = img[:, :, 1].reshape(-1, 1)
        B1 = img[:, :, 2].reshape(-1, 1)

        L, A, B = RGB2LAB2(R1, G1, B1)

        L = L.reshape(sz0, sz1, 1)
        A = A.reshape(sz0, sz1)
        B = B.reshape(sz0, sz1)

        ab = np.stack([A, B], axis=2)

        L_tensor = torch.FloatTensor(L).permute(2, 0, 1)  
        ab_tensor = torch.FloatTensor(ab).permute(2, 0, 1)  

        return L_tensor, ab_tensor


In [ ]:
def load_vgg16_weights(model):
    """Load pretrained VGG16 weights to U-Net encoder"""
    vgg16 = models.vgg16(pretrained=True).to(device)
    vgg_features = vgg16.features

    with torch.no_grad():
        rgb_weights = vgg_features[0].weight

        gray_weights = rgb_weights.mean(dim=1, keepdim=True)  

        # Set weights for first layer
        model.conv1.double_conv[0].weight.data = gray_weights
        model.conv1.double_conv[0].bias.data = vgg_features[0].bias.data

        # Set weights for second conv in first block
        model.conv1.double_conv[2].weight.data = vgg_features[2].weight.data
        model.conv1.double_conv[2].bias.data = vgg_features[2].bias.data

        # Second block
        model.conv2.double_conv[0].weight.data = vgg_features[5].weight.data
        model.conv2.double_conv[0].bias.data = vgg_features[5].bias.data
        model.conv2.double_conv[2].weight.data = vgg_features[7].weight.data
        model.conv2.double_conv[2].bias.data = vgg_features[7].bias.data

        # Third block (first two convs)
        model.conv3.triple_conv[0].weight.data = vgg_features[10].weight.data
        model.conv3.triple_conv[0].bias.data = vgg_features[10].bias.data
        model.conv3.triple_conv[2].weight.data = vgg_features[12].weight.data
        model.conv3.triple_conv[2].bias.data = vgg_features[12].bias.data
        model.conv3.triple_conv[4].weight.data = vgg_features[14].weight.data
        model.conv3.triple_conv[4].bias.data = vgg_features[14].bias.data

        # Fourth block
        model.conv4.triple_conv[0].weight.data = vgg_features[17].weight.data
        model.conv4.triple_conv[0].bias.data = vgg_features[17].bias.data
        model.conv4.triple_conv[2].weight.data = vgg_features[19].weight.data
        model.conv4.triple_conv[2].bias.data = vgg_features[19].bias.data
        model.conv4.triple_conv[4].weight.data = vgg_features[21].weight.data
        model.conv4.triple_conv[4].bias.data = vgg_features[21].bias.data

        # Fifth block
        model.conv5.triple_conv[0].weight.data = vgg_features[24].weight.data
        model.conv5.triple_conv[0].bias.data = vgg_features[24].bias.data
        model.conv5.triple_conv[2].weight.data = vgg_features[26].weight.data
        model.conv5.triple_conv[2].bias.data = vgg_features[26].bias.data
        model.conv5.triple_conv[4].weight.data = vgg_features[28].weight.data
        model.conv5.triple_conv[4].bias.data = vgg_features[28].bias.data


In [ ]:
def train_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Parameters
    # dim = 150 # For Intel Image
    dim = 256 # For Landscape Image
    batch_size = 16
    epochs_max = 10
    max_nb_min = 3

    cwd = os.getcwd()
    # train_path = os.path.join(cwd, 'seg_train', '*.png')
    # files_tr_list = glob.glob(train_path)

    # N = len(files_tr_list)
    # print(f'Number of training images: {N}')



    base_path = "/content/intel-image-classification/seg_train/seg_train"

    # For Intel-Image Dataset
    classes = os.listdir(base_path)

    image_paths = []

    for cls in classes:
        folder_path = os.path.join(base_path, cls, "*.jpg")
        for file in glob.glob(folder_path):
            image_paths.append(file)
            if len(image_paths) >= 3000:
                break
        if len(image_paths) >= 3000:
            break

    # For Landscape Dataset
    # image_paths = glob.glob(os.path.join(base_path, "*.jpg"))

    print("Number of training images:", len(image_paths))

    # Create model
    model = UNet1(in_channels=1, out_channels=2).to(device)

    print('Loading VGG16 pretrained weights...')
    load_vgg16_weights(model)

    # Loss function and optimizer
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    dataset = ColorizationDataset(image_paths, dim=dim)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    wandb.init(project="image_colorization", name="HyperUNet_experiment-Landscape-Image",
               config={
                   "batch_size": batch_size,
                   "epochs": epochs_max,
                   "learning_rate": 1e-4,
                   "model_name": "HyperUNet-Intel-Image",
                   "optimizer": "Adam",
                   "loss_function": "L1Loss",
                   "dataset": "Landscape"
               })

    tr_acc = np.zeros((epochs_max, 2))
    time_tr = np.zeros((epochs_max, 2))
    mae_min = float('inf')
    nb_min = 0
    stop = 0

    print('Starting training...')

    for epoch in range(epochs_max):
        if stop:
            break

        start_time = datetime.now()
        model.train()

        total_loss = 0.0
        num_batches = 0

        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            num_batches += 1

            # wandb.log({"batch_loss": loss.item()})

            if batch_idx % 10 == 0:
                print(f'Epoch {epoch+1}, Batch {batch_idx}/{len(dataloader)}, Loss: {loss.item():.6f}')

        avg_loss = total_loss / num_batches
        wandb.log({"epoch": epoch+1, "avg_loss": avg_loss})

        # Update tracking arrays
        tr_acc[epoch, 0] = epoch
        tr_acc[epoch, 1] = avg_loss

        end_time = datetime.now()
        time_diff = end_time - start_time
        time_tr[epoch, 0] = epoch
        time_tr[epoch, 1] = time_diff.seconds

        print(f'Epoch {epoch+1}/{epochs_max}, Average Loss: {avg_loss:.6f}, Time: {time_diff.seconds}s')

        # Early stopping logic
        if avg_loss > mae_min:
            nb_min += 1
        else:
            mae_min = avg_loss
            nb_min = 0
            # Save best model
            torch.save(model.state_dict(), os.path.join(cwd, 'Hyper_U_NET_pytorch.pth'))
            print(f'New best model saved with loss: {mae_min:.6f}')

        if nb_min > max_nb_min:
            stop = 1
            print('Early stopping triggered')

        # Learning rate scheduling 
        if epoch + 1 == 1:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 5e-5
        elif epoch + 1 == 2:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 2e-5
        elif epoch + 1 == 4:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 1e-5
        elif epoch + 1 == 8:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 5e-6
        elif epoch + 1 == 16:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 2e-6
        elif epoch + 1 == 32:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 1e-6
        elif epoch + 1 == 64:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 5e-7
        elif epoch + 1 == 128:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 2e-7
        elif epoch + 1 == 256:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 1e-7


        np.save(os.path.join(cwd, 'tr_Acc_Hyper_U_NET_pytorch.npy'), tr_acc)
        np.save(os.path.join(cwd, 'Tr_runtime_Hyper_U_NET_pytorch.npy'), time_tr)




In [ ]:
def load_model_for_inference(model_path, device):
    model = UNet1(in_channels=1, out_channels=2).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

def inference(model, l_channel):
    model.eval()
    with torch.no_grad():
        if len(l_channel.shape) == 3:
            l_channel = l_channel.unsqueeze(0)  

        l_tensor = torch.FloatTensor(l_channel).to(device)
        ab_pred = model(l_tensor)

        return ab_pred.cpu().numpy()


In [13]:
if __name__ == 'main':
  train_model()

### Optuna Hyperparameters

In [15]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 23.8 MB/s eta 0:00:00


In [ ]:
def objective(trial):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dim = trial.suggest_categorical("dim", [128, 150, 256])
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    epochs_max = 5
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)

    cwd = os.getcwd()
    base_path = "/content/intel-image-classification/seg_train/seg_train"
    classes = os.listdir(base_path)

    image_paths = []
    for cls in classes:
        folder_path = os.path.join(base_path, cls, "*.jpg")
        for file in glob.glob(folder_path):
            image_paths.append(file)
            if len(image_paths) >= 2000:
                break
        if len(image_paths) >= 2000:
            break

    print("Number of training images:", len(image_paths))

    # Model
    model = UNet1(in_channels=1, out_channels=2).to(device)
    load_vgg16_weights(model)

    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Dataset
    dataset = ColorizationDataset(image_paths, dim=dim)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    mae_min = float('inf')
    nb_min = 0
    max_nb_min = 2

    for epoch in range(epochs_max):

        model.train()
        total_loss = 0.0
        num_batches = 0

        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            num_batches += 1

        avg_loss = total_loss / num_batches

        if avg_loss > mae_min:
            nb_min += 1
        else:
            mae_min = avg_loss
            nb_min = 0

        if nb_min > max_nb_min:
            print("Early stopping triggered")
            break

    return mae_min


if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=10)

    print("Best hyperparameters:", study.best_params)
    print("Best loss:", study.best_value)


[I 2025-08-28 07:02:20,633] A new study created in memory with name: no-name-bf74153d-748a-407e-9a74-f4411eb5a266
/tmp/ipython-input-2584549285.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)


Number of training images: 2000


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:05<00:00, 104MB/s]
[I 2025-08-28 07:07:30,517] Trial 0 finished with value: 0.060671099364757536 and parameters: {'dim': 150, 'batch_size': 16, 'lr': 0.000593811464694052}. Best is trial 0 with value: 0.060671099364757536.


...

Number of training images: 2000


[I 2025-08-28 08:13:27,377] Trial 8 finished with value: 0.05722824528813362 and parameters: {'dim': 256, 'batch_size': 8, 'lr': 0.00014136731385907975}. Best is trial 7 with value: 0.055366841286420825.


Number of training images: 2000


[I 2025-08-28 08:20:36,895] Trial 9 finished with value: 0.05699840810894966 and parameters: {'dim': 150, 'batch_size': 8, 'lr': 0.00012005467161378065}. Best is trial 7 with value: 0.055366841286420825.


Best hyperparameters: {'dim': 128, 'batch_size': 16, 'lr': 5.6008593690527854e-05}
Best loss: 0.055366841286420825


hello
